# Symbolic Partial Derivative Routine

## Authors: Zach Etienne, Tyler Knowles & Siddharth Mahesh

## This module contains a routine for computing partial derivatives of a mathematical expression that is written as several subexpressions.

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). Additionally, this notebook has been validated by checking that results are consistent with exact derivative expressions used in the SEOBNRv3_opt approixment of [LALSuite](https://git.ligo.org/lscsoft/lalsuite).

### NRPy+ Source Code for this module: [SEOBNR_Derivative_Routine.py](../edit/SEOBNR/SEOBNR_Derivative_Routine.py)

## Introduction
$$\label{intro}$$

This notebook documents the symbolic partial derivative routine used to generate analytic derivatives of the [SEOBNRv3](https://git.ligo.org/lscsoft/lalsuite) Hamiltonian (documented [here](../Tutorial-SEOBNR_v3_Hamiltonian.ipynb)) and described in [this article](https://arxiv.org/abs/1803.06346).  In general, this notebook takes as input a file of inter-dependent mathematical expressions (in SymPy syntax), a file listing the names of values within those expressions, and a file listing all variables with which to take partial derivatives of each expression.  The output is a text file containing the original expression and those for each partial derivative computation.  The intention is to perform CSE on these expressions to create efficient partial derivative code!

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2:](#read_expressions) Read in Hamiltonian expressions from `Sympy_Hreal_on_Bottom.txt`
1. [Step 3:](#list_constants) Specify constants and variables in Hamiltonian expression
1. [Step 4:](#list_free_symbols) Extract free symbols
1. [Step 5:](#convert_to_func) Convert variables to function notation; e.g., `var` goes to `var(xx)`
1. [Step 6:](#differentiate) Differentiate with respect to `xx`
1. [Step 7:](#remove_zeros) Remove derivatives (of constants) that evaluate to zero, simplifying derivative expressions
1. [Step 8:](#store_results) Store partial derivatives to SymPy notebook `partial_derivatives.txt-VALIDATION.txt`
1. [Step 9:](#numpy_function) Create numpy function to output exact derivatives for all 12 dynamic variables 
1. [Step 10:](#code_validation) Validate against LALSuite and trusted `SEOBNR_Derivative_Routine` NRPy+ module
1. [Step 11:](#spherical_orbit) Spherical Orbit Test
1. [Step 11:](#latex_pdf_output) Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from Python/NRPy+ and creating the output directory (if it does not already exist). Note that since the expression 'Q' appears in the Hamiltonian and is a protected symbol in sympy we must create a new global dictionary to ensure that sympy does not spit out errors.

In [1]:
# Step 1.a: import all needed modules from Python/NRPy+:
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends
import sys, os     # Standard Python modules for multiplatform OS-level functions
sys.path.append('../')

from outputC import superfast_uniq, lhrh      # Remove duplicate entries from a Python array; store left- and right-
                                              #   hand sides of mathematical expressions

# As of April 2021, "sp.sympify("Q+1")" fails because Q is a reserved keyword.
#   This is the workaround, courtesy Ken Sible.
custom_global_dict = {}
exec('from sympy import *', custom_global_dict)
del custom_global_dict['Q']

# Step 1.b: Check for a sufficiently new version of SymPy (for validation)
# Ignore the rc's and b's for release candidates & betas.
sympy_version = sp.__version__.replace('rc', '...').replace('b', '...')
sympy_version_decimal = float(int(sympy_version.split(".")[0]) + int(sympy_version.split(".")[1])/10.0)
if sympy_version_decimal > 1.2:
    custom_parse_expr = lambda expr: sp.parse_expr(expr, global_dict=custom_global_dict)
else:
    custom_parse_expr = lambda expr: sp.sympify(expr)

if sympy_version_decimal < 1.2:
    print('Error: NRPy+ does not support SymPy < 1.2')
    sys.exit(1)

# Step 1.c: Name of the directory containing the input file
inputdir = "Hamiltonian"

<a id='read_expressions'></a>

# Step 2: Read in Hamiltonian expressions from `Sympy_Hreal_on_Bottom.txt` \[Back to [top](#toc)\]
$$\label{read_expressions}$$

We read in the expressions of which we will compute partial derivatives in a single large string before splitting the string by line (carriage return) and by "=".  Doing so allows us to manipulate the right- and left-hand sides of the expressions appropriately.  We store the left- and right-hand sides in the array `lr`, which consists of `lhrh` arrays with left-hand sides `lhs` and right-hand sides `rhs`.  Note that `Lambda` is a protected keyword in Python, so the variable $\Lambda$ in the Hamiltonian is renamed `Lamb`.

In [2]:
# Step 2.a: Read in expressions as a (single) string
with open(os.path.join(inputdir,'Sympy_Hreal_on_Bottom.txt'), 'r') as file:
    expressions_as_lines = file.readlines()

#print(expressions_as_lines)
# Step 2.b: Create and populate the "lr" array, which separates each line into left- and right-hand sides
#   Each entry is a string of the form lhrh(lhs='',rhs='')
lr = []

for i in range(len(expressions_as_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(expressions_as_lines[i]) > 2 and expressions_as_lines[i][0] != "#":
        # Split each line by its equals sign
        split_line = expressions_as_lines[i].split("=")
        #print(split_line)
        # Append the line to "lr", removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #   (Lambda is a protected keyword):
        lr.append(lhrh(lhs=split_line[0].replace(" ","").replace("Lambda","Lamb"),
                       rhs=split_line[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

# Step 2.c: Separate and sympify right- and left-hand sides into separate arrays
lhss = []
rhss = []
for i in range(len(lr)):
    #print(lr[i].rhs)
    #print(lr[i].rhs)
    lhss.append(custom_parse_expr(lr[i].lhs))
    rhss.append(custom_parse_expr(lr[i].rhs))

0.577215664901532860606512090082402431

m1+m2

m1*m2/M

mu/M

sqrt(x*x+y*y+z*z)

1/r

m2/m1*S1z+m1/m2*S2z

m2/m1*S1y+m1/m2*S2y

m2/m1*S1x+m1/m2*S2x

S1z+S2z

S1y+S2y

S1x+S2x

sigma3

sigma2

sigma1

sqrt(Skerr1*Skerr1+Skerr2*Skerr2+Skerr3*Skerr3)

Skerr3/Skerrmag

Skerr2/Skerrmag

Skerr1/Skerrmag

Skerrmag

x*p2-y*p1

z*p1-x*p3

y*p3-z*p2

sqrt(L1*L1+L2*L2+L3*L3)

L3/Lnorm

L2/Lnorm

L1/Lnorm

S2x*Lhat1+S2y*Lhat2+S2z*Lhat3

S1x*Lhat1+S1y*Lhat2+S1z*Lhat3

S1z-S1dotLhat*Lhat3

S1y-S1dotLhat*Lhat2

S1x-S1dotLhat*Lhat1

S2z-S2dotLhat*Lhat3

S2y-S2dotLhat*Lhat2

S2x-S2dotLhat*Lhat1

S1perp3+S2perp3

S1perp2+S2perp2

S1perp1+S2perp1

z/r

y/r

x/r

Skerrhat3

Skerrhat2

Skerrhat1

e31*n2-e32*n1

-e31*n3+e33*n1

e32*n3-e33*n2

n1*xi2-n2*xi1

n3*xi1-n1*xi3

n2*xi3-n3*xi2

e31*n1+e32*n2+e33*n3

1-costheta*costheta

sin2theta

a*a+r*r

r*r+a*a*costheta*costheta

1+log(1+6*eta*u*u+2*(26-3*eta)*eta*u*u*u)

2*a*r

(Skerr1*Lhat1+Skerr2*Lhat2+Skerr3*Lhat3)/(1-2*eta)+Rational(1,2)*(Sperp1*Skerr1+Sper

<a id='list_constants'></a>

# Step 3: Specify constants and variables in Hamiltonian expression \[Back to [top](#toc)\]
$$\label{list_constants}$$

We read in and declare as SymPy symbols the constant values; derivatives with respect to these variables will be set to zero.  We then read in the variables with respect to which we want to take derivatives and declare those as SymPy variables as well.

In [3]:
# Step 3.a: Create `input_constants` array and populate with SymPy symbols
m1,m2,tortoise,eta,EMgamma = sp.symbols('m1 m2 tortoise eta EMgamma',
                                                                real=True)
input_constants = [m1,m2,tortoise,eta,EMgamma]

# Step 3.b: Create `dynamic_variables` array and populate with SymPy symbols
#x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z = sp.symbols("x y z p1 p2 p3 S1x S1y S1z S2x S2y S2z", real=True)
x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z = sp.symbols("x y z p1 p2 p3 S1x S1y S1z S2x S2y S2z")
dynamic_variables = [x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z]

<a id='list_free_symbols'></a>

# Step 4: Extract free symbols \[Back to [top](#toc)\]
$$\label{list_free_symbols}$$

By ''free symbols'' we mean the variables in the right-hand sides.  We first create a list of all such terms (using SymPy's built-in free_symbol attribute), including duplicates, and then strip the duplicates.  We then remove input constants from the symbol list.

In [4]:
# Step 4.a: Prepare array of "free symbols" in the right-hand side expressions
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for variable in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(variable)

# Step 4.b: Remove duplicate free symbols
full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

# Step 4.c: Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)
for dynamic_variable in dynamic_variables:
    for symbol in full_symbol_list:
        if str(symbol) == str(dynamic_variable):
            full_symbol_list.remove(symbol)


<a id='convert_to_func'></a>

# Step 5: Convert variables to function notation; e.g., `var` goes to `var(x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z)` \[Back to [top](#toc)\]
$$\label{convert_to_func}$$

In order to compute the partial derivative of each right-hand side, we mark each variable (left-hand side) and each free symbol (in right-hand sides) as a function with the dynamic variables as arguments.

In [5]:
# Step 5.b: Mark each free variable as a function with arguments as the dynamic variables
full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb),real=True)(x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)

<a id='differentiate'></a>

# Step 6: Differentiate with respect to dynamic variables \[Back to [top](#toc)\]
$$\label{differentiate}$$

Now we differentiate the right-hand expressions with respect to the dynamic variabls.  We use the SymPy $\texttt{diff}$ command, differentiating with respect to each dynamic variable.  After so doing, we remove $\texttt{(xx)}$, where 'xx' represents a dynamic variable and "Derivative" (which is output by $\texttt{diff}$), and use "prm_xx" suffix to denote the derivative with respect to $\texttt{xx}$.

In [6]:
# Step 6: Use SymPy's diff function to differentiate right-hand sides with respect to dynamic variables
#   and append "prm" notation to left-hand sides
lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
    for dynamic_variable in dynamic_variables:
        var = str(dynamic_variable)
        lhss_deriv.append(custom_parse_expr(str(lhss[i])+"prm"+var))
        newrhs = custom_parse_expr(str(sp.diff(rhss[i],dynamic_variable)).replace("(x, y, z, p1, p2, p3, S1x, S1y, S1z, S2x, S2y, S2z)","").replace(", "+var+")","prm"+var).replace("Derivative(",""))
        rhss_deriv.append(newrhs)

<a id='remove_zeros'></a>

# Step 7: Remove derivatives (of constants) that evaluate to zero, simplifying derivative expressions \[Back to [top](#toc)\]
$$\label{remove_zeros}$$

We declare a function to simply the derivative expressions.  In particular, we want to remove terms equal to zero.

In [7]:
# Step 7.a: Define derivative simplification function
def simplify_deriv(lhss_deriv,rhss_deriv):
    # Copy expressions into another array
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    # If a right-hand side is 0, substitute value 0 for the corresponding left-hand side in later terms
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    # Create array of indices for expressions that are zero
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)
    # When removing terms that are zero, we need to take into account their new index (after each removal)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

# Step 7.b: Call the simplication function and then copy results
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

<a id='store_results'></a>

# Step 8: Store partial derivatives to SymPy notebook `partial_derivatives.txt-VALIDATION.txt` \[Back to [top](#toc)\]
$$\label{store_results}$$

We write the resulting derivatives in SymPy syntax.  Each partial derivative is output in its own file, in a similar format to the input expressions.

In [8]:
# Step 9: Output original expression and each partial derivative expression in SymPy snytax
outputdir = "Derivatives"
with open(os.path.join(outputdir,'partial_derivatives.txt-VALIDATION'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    for i in range(len(lhss_deriv)):
            right_side = str(rhss_deriv[i])
            right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(").replace("prm",
                                                "_prm")
            output.write(str(lhss_deriv[i]).replace("prm","_prm")+" = "+right_side_in_sp+"\n")

<a id='numpy_function'></a>

# Step 9: Create numpy function to output exact derivatives for all 12 dynamic variables \[Back to [top](#toc)\]
$$\label{numpy_function}$$

We create a python module where we store the exact derivatives as a numpy function.

In [16]:
# Step 10: Create Numpy function to output exact derivatives for all 12 dynamical variables

with open(os.path.join(outputdir,'hamiltonian_first_derivatives.py'),"w") as file:
    file.write("""from __future__ import division
import numpy as np
def ham_first_derivs(m1, m2, tortoise, x, y, z, p1, p2, p3, S1x, S1y, S1z, S2x, S2y, S2z):
""")
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_np = right_side.replace("sqrt(","np.sqrt(").replace("log(","np.log(").replace("log(1+","log1p(").replace("pi",
                                                "np.pi").replace("sign(","np.sign(").replace("Abs(",
                                                "np.abs(").replace("Rational(","np.true_divide(")
        file.write("    " + lr[i].lhs + " = " + right_side_in_np)
    for i in range(len(lhss_deriv)):
        right_side = str(rhss_deriv[i])
        right_side_in_np = right_side.replace("sqrt(","np.sqrt(").replace("log(","np.log(").replace("log(1+","log1p(").replace("pi",
                                                "np.pi").replace("sign(","np.sign(").replace("Abs(",
                                                "np.abs(").replace("Rational(","np.true_divide(").replace("prm",
                                                "_prm")
        file.write("    " + str(lhss_deriv[i]).replace("prm", "_prm") + " = " + right_side_in_np + "\n")
    file.write("    return np.array([Hreal_prmx, Hreal_prmy, Hreal_prmz, Hreal_prmp1, Hreal_prmp2, Hreal_prmp3, Hreal_prmS1x, Hreal_prmS1y, Hreal_prmS1z, Hreal_prmS2x, Hreal_prmS2y, Hreal_prmS2z])\n")

with open(os.path.join(outputdir,'v4P_hamiltonian_first_derivatives.c'),"w") as file:
    file.write("#include math.h\nint ham_first_derivs(REAL8 dvalues[],REAL8 m1,REAL8 m2,INT tortoise,REAL8 x,REAL8 y,REAL8 z,REAL8 p1,REAL8 p2,REAL8 p3,REAL8 S1x,REAL8 S1y,REAL8 S1z,REAL8 S2x,REAL8 S2y,REAL8 S2z){\n")
    for i in range(len(lr)):
        right_side = str(lr[i].rhs).strip() + ";\n"
        right_side_in_c = right_side.replace("sqrt(","sqrt(").replace("log(","log(").replace("log(1+","log1p(").replace("pi",
                                                "M_PI").replace("sign(","copysign(").replace("Abs(",
                                                "fabs(").replace("Rational(","np.true_divide(").replace("tortoise","(REAL8)tortoise")
        file.write("    REAL8 " + lr[i].lhs + " = " + right_side_in_c)
    for i in range(len(lhss_deriv)):
        right_side = str(rhss_deriv[i]).strip() + ";\n"
        right_side_in_c = right_side.replace("sqrt(","sqrt(").replace("log(","log(").replace("log(1+","log1p(").replace("pi",
                                                "M_PI").replace("sign(","copysign(").replace("Abs(",
                                                "fabs(").replace("Rational(","np.true_divide(").replace("prm",
                                                "_prm").replace("tortoise","(REAL8)tortoise")
        file.write("    REAL8 " + str(lhss_deriv[i]).replace("prm", "_prm") + " = " + right_side_in_c)
    file.write("    dvalues[0] = Hreal_prmx;\n"+
                   "    dvalues[1] = Hreal_prmy;\n"+
                   "    dvalues[2] = Hreal_prmz;\n"+
                   "    dvalues[3] = Hreal_prmp1;\n"+
                   "    dvalues[4] = Hreal_prmp2;\n"+
                   "    dvalues[5] = Hreal_prmp3;\n"+
                   "    dvalues[6] = Hreal_prmS1x;\n"+
                   "    dvalues[7] = Hreal_prmS1y;\n"+
                   "    dvalues[8] = Hreal_prmS1z;\n"+ 
                   "    dvalues[9] = Hreal_prmS2x;\n"+
                   "    dvalues[10] = Hreal_prmS2y;\n"+
                   "    dvalues[11] = Hreal_prmS2z;\n"+
                   "    return 1;\n}")
    
    
with open(os.path.join(outputdir,'ham_and_csi_first_derivs.py'),"w") as file:
    file.write("""from __future__ import division
import numpy as np
def ham_first_derivs(m1, m2, tortoise, x, y, z, p1, p2, p3, S1x, S1y, S1z, S2x, S2y, S2z):
""")
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_np = right_side.replace("sqrt(","np.sqrt(").replace("log(","np.log(").replace("log(1+","log1p(").replace("pi",
                                                "np.pi").replace("sign(","np.sign(").replace("Abs(",
                                                "np.abs(").replace("Rational(","np.true_divide(")
        file.write("    " + lr[i].lhs + " = " + right_side_in_np)
    for i in range(len(lhss_deriv)):
        right_side = str(rhss_deriv[i])
        right_side_in_np = right_side.replace("sqrt(","np.sqrt(").replace("log(","np.log(").replace("log(1+","log1p(").replace("pi",
                                                "np.pi").replace("sign(","np.sign(").replace("Abs(",
                                                "np.abs(").replace("Rational(","np.true_divide(").replace("prm",
                                                "_prm")
        file.write("    " + str(lhss_deriv[i]).replace("prm", "_prm") + " = " + right_side_in_np + "\n")
    file.write("    return np.array([Hreal_prmx, Hreal_prmy, Hreal_prmz, Hreal_prmp1, Hreal_prmp2, Hreal_prmp3, Hreal_prmS1x, Hreal_prmS1y, Hreal_prmS1z, Hreal_prmS2x, Hreal_prmS2y, Hreal_prmS2z, csi_prmx, csi_prmy, csi_prmz])\n")
    

<a id='code_validation'></a>

# Step 10: Validate against LALSuite and trusted `SEOBNR_Derivative_Routine` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

We validate the output of this notebook against known LALSuite values of the Hamiltonian partial derivatives and the output of the `SEOBNR_Derivative_Routine` NRPy+ module.  We note that due to cancellations in the deriavtive terms, various versions of SymPy may result in relative errors that differ as much as an order of magnitude.  Furthermore, even changing the set of input pararameters can affect the relative error by as many as two orders of magnitude.  Therefore we look for agreement with LALSuite to at least 10 significant digits.

When comparing the notebook output to that of the NRPy+ module, we compare term-by-term using SymPy to check that each right-hand side side is equivalent.

In [ ]:
## Validate against LALSuite
import numpy as np
from Derivatives.hamiltonian_first_derivatives import ham_first_derivs as dH

for index in range(3):
    print("for index %d"%index)
    validationfile = 'outputv4Pindex'+str(index)+'.txt'
    validationfile_pert = 'outputv4Pindex'+str(index)+'pert.txt'
    with open(os.path.join(outputdir,validationfile)) as datfile, open(os.path.join(outputdir,validationfile_pert)) as pertdatfile:
        data_lines = datfile.readlines()
        pertdata_lines = pertdatfile.readlines()
        for i in range(len(pertdata_lines)//3):
            valuepairs = data_lines[3*i].strip('{}\n').split(',')
            valuepairspert = pertdata_lines[3*i].strip('{}\n').split(',')
            derivpairs = data_lines[3*i+1].strip('{}\n').split(',')
            derivpairspert = pertdata_lines[3*i+1].strip('{}\n').split(',')
            values = dict()
            derivs = dict()
            valuespert = dict()
            derivspert = dict()
            for j in range(len(valuepairs)):
                pair = valuepairs[j].split(': ')
                pairpert = valuepairspert[j].split(': ')
                values[pair[0].strip(" ''")] = float(pair[1])
                valuespert[pairpert[0].strip(" ''")] = float(pairpert[1])
            for j in range(len(derivpairs)):
                pair = derivpairs[j].split(': ')
                pairpert = derivpairspert[j].split(': ')
                derivs[pair[0].strip(" ''")] = float(pair[1])
                derivspert[pairpert[0].strip(" ''")] = float(pairpert[1])
            
            m1 , m2 = values['m1'], values['m2']
            eta = m1*m2/(m1 + m2)/(m1 + m2)
            
            nrpy_derivs = dH(values['m1'],values['m2'],values['tortoise'],values['x'],values['y'],values['z'],values['p1'],values['p2'],values['p3'],values['S1x'],values['S1y'],values['S1z'],values['S2x'],values['S2y'],values['S2z'])
            for j in range(len(dynamic_variables)):
                var = str(dynamic_variables[j])
                print(var,derivs['dHreal_d'+var])
                if(derivs['dHreal_d'+var] == 0):
                    Erel = np.abs(nrpy_derivs[j]/eta - derivs['dHreal_d'+var])
                    Erelpert = np.abs(derivs['dHreal_d'+var]-derivspert['dHreal_d'+var])
                    if Erel > Erelpert and Erel > 1e-15:
                        print("in line %d: abs error in %s derivative is %.2e, allowed is %.2e"%(i,var,Erel,Erelpert))
                else:
                    Erel = np.abs((nrpy_derivs[j]/eta - derivs['dHreal_d'+var])/derivs['dHreal_d'+var])
                    Erelpert = np.abs((derivs['dHreal_d'+var]-derivspert['dHreal_d'+var])/derivs['dHreal_d'+var])
                    if Erel > Erelpert:
                        if Erelpert != 0:
                            print("In line %d: ratio of Erel to allowed in %s derivative is %.2e"%(i,var,Erel/Erelpert))
                        else:
                            print("in line %d: abs error in %s derivative is %.2e, allowed is %.2e"%(i,var,Erel,Erelpert))
            
        
    


<a id='spherical_orit'></a>

# Step 11: Spherical Orbit Test  \[Back to [top](#toc)\]
$$\label{spherical_orbit}$$

In this test we construct a spherical orbit for a set of initial conditions and then perturb the initial conditions to assess a measure of intrinsic error.

In [ ]:
from scipy.optimize import root

m1 = 23.
m1pert = 23.*(1. + 1.e-15)
m2 = 10.
f = 20.
S1 = np.array([0.0, 0.0, -0.03])
S2 = np.array([0.0, 0.0, 0.06])


EMgamma = 0.577215664901532860606512090082402431
# Geomtrized solar mass $\mathcal{M}_{\odot}$
Msol = 4.925491025543575903411922162094833998e-6

#Convert the spins to dimensionless quantities
S1 *= m1*m1
S2 *= m2*m2

# Binary system total mass $M$
M = m1 + m2
Mpert = m1pert + m2

# Inverse mass terms used repeatedly when computing initial conditions
Minv = np.divide(1,M)
Minvpert = np.divide(1,M)
Msqinv = Minv*Minv
Msqinvpert = Minvpert*Minvpert

# Symmetric mass ratio $\eta$
eta = m1*m2*Msqinv
etapert = m1pert*m2*Msqinvpert

# Normalized, dimensionless spin vectors
S1hat = Msqinv*S1
S2hat = Msqinv*S2

S1hatpert = Msqinvpert*S1
S2hatpert = Msqinvpert*S2

# Omega: initial orbital angular frequency
omega = M*Msol*np.pi*f
omegapert = Mpert*Msol*np.pi*f

v = np.cbrt(omega)
vpert = np.cbrt(omegapert)
vsq = v*v
vsqpert = vpert*vpert

# Inital root guess
root_guess = [np.divide(1,v*v), v*2, 0.]
root_guesspert = [np.divide(1,vpert*vpert),vpert*2,0.]

# This is the same initial guess given to GSL in LALSuite

# Define the function of which we want to find the roots
def root_func(F,m1,m2,S1hat,S2hat,omega,eta):
    #Recompute Hamiltonian derivatives using latest minimization guess
    Ham_derivs = dH(m1, m2, 2, F[0], 0.0, 0.0, 0.0, F[1], F[2], S1hat[0], S1hat[1], S1hat[2], S2hat[0],
                          S2hat[1], S2hat[2])
    return [Ham_derivs[0]/eta+(-Ham_derivs[5]*F[2]/eta-Ham_derivs[4]*F[1]/eta)/F[0], -Ham_derivs[5]/F[0]/eta, Ham_derivs[4]/F[0]/eta-omega]

soln = root(root_func, root_guess, args=(m1,m2,S1hat,S2hat,omega,eta), method='hybr', jac=None, tol=None, callback=None)
solnpert = root(root_func,root_guesspert, args = (m1pert,m2,S1hatpert,S2hatpert,omegapert,etapert), method='hybr', jac=None, tol=None, callback=None)

if not(soln.success):
    print("The root finder failed with error message: %s" % soln.message)
    sys.exit(1)

r, py , pz = soln.x[0] , soln.x[1], soln.x[2]
rpert, pypert, pzpert = solnpert.x[0] , solnpert.x[1] , solnpert.x[2]

Erel = [np.abs((r - rpert)/r), np.abs((py - pypert)/py), np.abs((pz - pzpert))]


LAL_init_conds = {'r': 2.1296908267405176e+01, 'py': 2.3354017824522000e-01, 'pz': 0.0000000000000000e+00}
LAL_init_condspert = {'r': 2.1296908268016757e+01, 'py': 2.3354017824291295e-01, 'pz': 0.0000000000000000e+00}

Erel_LAL = [np.abs((LAL_init_conds['r'] - LAL_init_condspert['r'])/LAL_init_conds['r']), np.abs((LAL_init_conds['py'] - LAL_init_condspert['py'])/LAL_init_conds['py']), np.abs((LAL_init_conds['pz'] - LAL_init_condspert['pz']))]

print(r,py,pz)

labels = ['r','py','pz']
for i in range(3):
    print("Self-agreement in %s in nrpy is %.3e and in LAL is %.3e"%(labels[i],Erel[i],Erel_LAL[i]))
    
print("Erel in radius between LAL and nrpy is %.3e" % np.abs((LAL_init_conds['r'] - r)/(0.5*(LAL_init_conds['r'] + r))))
print("Erel in azimuthal momentum between LAL and nrpy is %.3e" % np.abs((LAL_init_conds['py'] - py)/(0.5*(LAL_init_conds['py'] + py))))


# Step 12: Derivatives for Initial Conditions

In this section we use the above first derivatives to generate the relevant first and second derivatives for initial conditions



In [ ]:
ic_sd_vars = [x,py,pz]

outputdirinitconds = "InitialConditions"
lhss_initconds_derivs = []
rhss_initconds_derivs = []
with open(os.path.join(outputdir,'partial_derivatives_initconds.txt-VALIDATION'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    for i in range(len(lhss_deriv)):
        if(("prmx" in str(lhss_deriv[i])) or ("prmp2" in str(lhss_deriv[i])) or ("prmp3" in str(lhss_deriv[i]))):
            right_side = str(rhss_deriv[i])
            lhss_initconds_derivs.append(str(lhss_deriv[i]))
            rhss_initconds_derivs.append(str(rhss_deriv[i]))
            right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(").replace("prm",
                                                "_prm")
            output.write(str(lhss_deriv[i]).replace("prm","_prm")+" = "+right_side_in_sp+"\n")

